In [3]:
!pip install "Elasticsearch==8.7.0"
!pip install sqlalchemy
!pip install psycopg2-binary

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 11.2 MB/s eta 0:00:0000:0100:01


In [44]:
import pandas as pd
from sqlalchemy import create_engine

# Conexión a PostgreSQL
user = "userGESTDB"
password = "passGESTDB"
host = "postgres_db"  # nombre del servicio en docker-compose
port = "5432"
db = "GESTDB"

engine = create_engine(f"postgresql://{user}:{password}@{host}:{port}/{db}")

# Leer una tabla completa
df = pd.read_sql("SELECT * FROM grado;", engine)
df.head()

,id,nombre,id_area,descripcion,salidas
0,1,Grado en Ingeniería Informática,1,"Formación en programación, sistemas y redes","Desarrollador, analista, consultor IT"
1,2,Grado en Matemáticas,2,Formación avanzada en álgebra y estadística,"Docencia, investigación, finanzas"
2,3,Grado en Biología,3,Estudio de organismos vivos y ecosistemas,"Investigador, docente, técnico de laboratorio"
3,4,Grado en Derecho,4,Formación jurídica en distintas ramas del Derecho,"Abogacía, judicatura, asesoría legal"
4,5,Grado en Economía,5,Análisis económico y financiero,"Economista, analista, consultor"


In [45]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://elasticsearch:9200")

# Verifica la conexión
print(es.info().body)

{'name': '2e4dd1d218b7', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'EEfTU5MFTH2Fc3SU6wXKkw', 'version': {'number': '8.7.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09520b59b6bc1057340b55750186466ea715e30e', 'build_date': '2023-03-27T16:31:09.816451435Z', 'build_snapshot': False, 'lucene_version': '9.5.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


In [46]:
df.fillna({
        'nombre': 'Unknown', 
        'descripcion':'Este grado no contiene descripción',
        'salidas':'Este grado no contiene salidas'
        },
    inplace=True)

In [47]:
mapping = {
    "mappings": {
        "properties": {
            "id": { "type": "integer" },        
            "nombre": { "type": "keyword" },     
            "descripcion": { "type": "text" },
            "id_area": { "type": "integer" },
            "salidas": { "type": "text" },
        }
    }
}

index_name = 'salidas'
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, mappings=mapping["mappings"])
print(f"Index '{index_name}' created with mapping.")

Index 'salidas' created with mapping.


In [48]:
import json
from elasticsearch.helpers import bulk

def bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for doc in batch:
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

salidas = df.to_dict(orient='records')
bulk_index_data(es, salidas, index_name)


Indexed: 10 Errors: []


In [49]:
import json

# Query 1

res = es.search(
    index="salidas",
    size=3,
    query={
        "term": {
            "nombre": "Grado en Derecho"
        }
    }
)
print(json.dumps(res.body, indent=4))

{
    "took": 21,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 1,
            "relation": "eq"
        },
        "max_score": 1.9924302,
        "hits": [
            {
                "_index": "salidas",
                "_id": "4",
                "_score": 1.9924302,
                "_source": {
                    "id": 4,
                    "nombre": "Grado en Derecho",
                    "id_area": 4,
                    "descripcion": "Formaci\u00f3n jur\u00eddica en distintas ramas del Derecho",
                    "salidas": "Abogac\u00eda, judicatura, asesor\u00eda legal"
                }
            }
        ]
    }
}


In [50]:
# Query
res = es.search(
    index="salidas",
    size=3,
    query={
        "match": {
            "salidas": "docencia",
            "salidas": "investigación"
        }
    }
)
print(json.dumps(res.body, indent=4))

{
    "took": 13,
    "timed_out": false,
    "_shards": {
        "total": 1,
        "successful": 1,
        "skipped": 0,
        "failed": 0
    },
    "hits": {
        "total": {
            "value": 2,
            "relation": "eq"
        },
        "max_score": 1.5900146,
        "hits": [
            {
                "_index": "salidas",
                "_id": "2",
                "_score": 1.5900146,
                "_source": {
                    "id": 2,
                    "nombre": "Grado en Matem\u00e1ticas",
                    "id_area": 2,
                    "descripcion": "Formaci\u00f3n avanzada en \u00e1lgebra y estad\u00edstica",
                    "salidas": "Docencia, investigaci\u00f3n, finanzas"
                }
            },
            {
                "_index": "salidas",
                "_id": "8",
                "_score": 1.5900146,
                "_source": {
                    "id": 8,
                    "nombre": "Grado en Filolog\u00eda Hi

In [51]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.8/255.8 kB 6.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 16.5 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 353.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 10.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 13.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 14.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 18.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━

In [54]:
# indice semántico

vector_index_mapping = {
    "properties": {
        "id": { "type": "integer" },        
        "nombre": { "type": "keyword" },     
        "id_area": { "type": "integer" },
        "salidas": { "type": "text" },
        "descripcion": { "type": "text" },
        "descripcion_vector": {
            "type": "dense_vector",
            "dims": 384,  # Dimensionalityof the embeddings
            "index": True,
            "similarity": "cosine"
        }
    }
}

vector_index_name = "vector_descripcion"
if es.indices.exists(index=vector_index_name):
    es.indices.delete(index=vector_index_name)
es.indices.create(
    index=vector_index_name,
    mappings=vector_index_mapping
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector_descripcion'})

In [53]:
# creamos los embedings de la descripcion de titulo
from sentence_transformers import SentenceTransformer

# Load a pre-trained Sentence Transformer model
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Generate embeddings (vector representations) for the sentences
embeddings = model.encode(df["descripcion"].values.tolist())

# Print the vector for the first sentence
print(embeddings[0])

/opt/conda/lib/python3.8/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[ 8.35116059e-02 -1.93477824e-01 -1.19995192e-01 -1.29384592e-01
 -1.06979504e-01 -4.60089222e-02 -1.75135985e-01 -6.38076514e-02
 -4.90381643e-02  2.20434755e-01  7.85367563e-04  1.11985482e-01
  1.83193654e-01 -2.67934710e-01 -1.30328447e-01 -1.82548035e-02
 -6.06697127e-02 -5.49621694e-02  6.40818328e-02 -3.36430073e-01
 -1.35664135e-01 -3.71158481e-01 -2.14743808e-01 -1.71099141e-01
  1.07143983e-01 -2.43642293e-02 -8.77524614e-02 -6.04070202e-02
  1.82290018e-01 -3.34531844e-01  3.32695544e-02 -7.63322935e-02
  4.54823405e-01  2.66411547e-02 -2.50274092e-01  2.64493138e-01
  2.00054020e-01  2.93038309e-01 -2.95555294e-01 -9.57646742e-02
 -7.54416212e-02  1.23598889e-01  1.09403849e-01  1.30007610e-01
 -3.55219468e-02 -2.58114487e-01 -2.50911415e-01  1.71826240e-02
 -2.73760229e-01 -3.29264820e-01 -8.87088925e-02 -1.29653499e-01
  2.31072873e-01 -2.37689331e-01  1.24912776e-01 -1.48434192e-01
  2.42354825e-01  3.40424776e-01 -1.48256093e-01  7.47344047e-02
  2.12111786e-01 -1.26612

In [67]:
import json
from elasticsearch.helpers import bulk

def vector_bulk_index_data(es, data, index_name):
    batch_size = 50  # Reducir el tamaño del lote a 50
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        actions = []
        for idx, doc in enumerate(batch):
            doc["descripcion_vector"]=embeddings[i + idx].tolist()
            actions.append({
                "_index": index_name,
                "_id": doc['id'],
                "_source": doc
            })
        # Capturar la respuesta para verificar errores
        resp = bulk(es, actions, raise_on_error=True)
        print("Indexed:", resp[0], "Errors:", resp[1])

descripciones = df.to_dict(orient='records')
vector_bulk_index_data(es, descripciones, vector_index_name)

Indexed: 10 Errors: []


In [69]:
# Query semantica

query_sentence = "nuevas tecnologías"
query_vector = model.encode([query_sentence])[0]

parameters = {
     "field":"descripcion_vector",
     "query_vector": query_vector,
     "k":5,
     "num_candidates":100
}
res = es.search(
    index=vector_index_name, 
    knn=parameters)

print("Search Results:")
for hit in res['hits']['hits']:
     print(f"Document ID: {hit['_id']}, Nombre Título: {hit['_source']['nombre']}, Description: {hit['_source']['descripcion']}, Score: {hit['_score']}")

Search Results:
Document ID: 10, Nombre Título: Grado en Periodismo, Description: Comunicación informativa en medios, Score: 0.71908754
Document ID: 1, Nombre Título: Grado en Ingeniería Informática, Description: Formación en programación, sistemas y redes, Score: 0.71711886
Document ID: 6, Nombre Título: Grado en Arquitectura, Description: Diseño y construcción de edificaciones, Score: 0.66165257
Document ID: 2, Nombre Título: Grado en Matemáticas, Description: Formación avanzada en álgebra y estadística, Score: 0.6283113
Document ID: 7, Nombre Título: Grado en Psicología, Description: Estudio del comportamiento humano, Score: 0.60348797
